### Importing Libraries

In [1]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
def extract_features(wav_file_name):
    y, sr = librosa.load(wav_file_name)
    feature = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return feature

In [3]:
speech_labels = []
speech_data = []
for dirname, _, filenames in os.walk('ravdess speech data/'):
    for filename in filenames:
        label = int(filename[7:8]) - 1
        wav_file_name = os.path.join(dirname, filename)
        mfcc_features = extract_features(wav_file_name)
        speech_labels.append(label)
        speech_data.append(mfcc_features)

In [4]:
X = np.array(speech_data)
Y = to_categorical(np.array(speech_labels))

In [5]:
no_of_datapoints = X.shape[0]
train = int(no_of_datapoints*0.8)
valid = int(no_of_datapoints*0.1) # validation data
test = int(no_of_datapoints*0.1)

In [6]:
def create_model_LSTM():
    model = Sequential()
    model.add(LSTM(128, return_sequences=False, input_shape=(40, 1)))
    model.add(Dense(64))
    model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Dropout(0.4))
    model.add(Activation('relu'))
    model.add(Dense(8))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [7]:
train_data = np.expand_dims(X[:train],-1)
train_labels = Y[:train]

validation_data = np.expand_dims(X[train:train+valid],-1)
validation_labels = Y[train:train+valid]

In [8]:
model = create_model_LSTM()
history = model.fit(train_data,train_labels,
                    validation_data=(validation_data,validation_labels),
                    epochs=150,
                    shuffle=True)


Epoch 1/150


63/63 [==============================] - 11s 70ms/step - loss: 2.0581 - accuracy: 0.1642 - val_loss: 2.0354 - val_accuracy: 0.2063
Epoch 2/150
63/63 [==============================] - 3s 45ms/step - loss: 2.0076 - accuracy: 0.2014 - val_loss: 1.9733 - val_accuracy: 0.2183
Epoch 3/150
63/63 [==============================] - 3s 44ms/step - loss: 1.9471 - accuracy: 0.2302 - val_loss: 1.9315 - val_accuracy: 0.2222
Epoch 4/150
63/63 [==============================] - 3s 44ms/step - loss: 1.9131 - accuracy: 0.2599 - val_loss: 1.8834 - val_accuracy: 0.2778
Epoch 5/150
63/63 [==============================] - 3s 44ms/step - loss: 1.8760 - accuracy: 0.2713 - val_loss: 1.8683 - val_accuracy: 0.2857
Epoch 6/150
63/63 [==============================] - 3s 42ms/step - loss: 1.8656 - accuracy: 0.2817 - val_loss: 1.8453 - val_accuracy: 0.2897
Epoch 7/150
63/63 [==============================] - 3s 42ms/step - loss: 1.8355 - accuracy: 0.2892 - val_loss: 1.8771 - val_accuracy: 0.2976
Ep

Epoch 54/150
63/63 [==============================] - 3s 44ms/step - loss: 1.0094 - accuracy: 0.6235 - val_loss: 0.9477 - val_accuracy: 0.6587
Epoch 55/150
63/63 [==============================] - 3s 44ms/step - loss: 0.9842 - accuracy: 0.6344 - val_loss: 0.8608 - val_accuracy: 0.6746
Epoch 56/150
63/63 [==============================] - 3s 45ms/step - loss: 0.9133 - accuracy: 0.6587 - val_loss: 0.8749 - val_accuracy: 0.6865
Epoch 57/150
63/63 [==============================] - 3s 43ms/step - loss: 0.9106 - accuracy: 0.6711 - val_loss: 0.8600 - val_accuracy: 0.6627
Epoch 58/150
63/63 [==============================] - 3s 43ms/step - loss: 0.8855 - accuracy: 0.6736 - val_loss: 0.8262 - val_accuracy: 0.6905
Epoch 59/150
63/63 [==============================] - 3s 43ms/step - loss: 0.9207 - accuracy: 0.6637 - val_loss: 0.9293 - val_accuracy: 0.6468
Epoch 60/150
63/63 [==============================] - 3s 43ms/step - loss: 0.8843 - accuracy: 0.6687 - val_loss: 0.8375 - val_accuracy: 0.6825

In [9]:
model.evaluate(validation_data,validation_labels)

8/8 [==============================] - 0s 21ms/step - loss: 0.0289 - accuracy: 1.0000


[0.028878450393676758, 1.0]

In [10]:
model.save('modelspeech.h5')

C:\Users\Acer\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
